In [ ]:
!pip install keras_preprocessing
import numpy as np
import matplotlib as plt
import pandas as pd
import csv
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import gensim
from gensim.models import KeyedVectors
from gensim.models import word2vec
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00


In [ ]:
# downloading and unzipping n-gram model of 300 vectors by AraVec
!wget --load-cookies /tmp/cookies.txt "https://ia902803.us.archive.org/15/items/full_grams_cbow_300_twitter/full_grams_sg_300_twitter.zip" -O /content/full_uni_cbow_300_twitter.zip && rm -rf /tmp/cookies.txt
!unzip /content/full_uni_cbow_300_twitter.zip

Cannot open cookies file ‘/tmp/cookies.txt’: No such file or directory
--2023-08-10 10:00:29--  https://ia902803.us.archive.org/15/items/full_grams_cbow_300_twitter/full_grams_sg_300_twitter.zip
Resolving ia902803.us.archive.org (ia902803.us.archive.org)... 207.241.232.103
Connecting to ia902803.us.archive.org (ia902803.us.archive.org)|207.241.232.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3322543144 (3.1G) [application/zip]
Saving to: ‘/content/full_uni_cbow_300_twitter.zip’

/content/full_uni_c 100%[===================>]   3.09G  12.3MB/s    in 4m 54s  

2023-08-10 10:05:23 (10.8 MB/s) - ‘/content/full_uni_cbow_300_twitter.zip’ saved [3322543144/3322543144]

Archive:  /content/full_uni_cbow_300_twitter.zip
  inflating: full_grams_sg_300_twitter.mdl  
  inflating: full_grams_sg_300_twitter.mdl.trainables.syn1neg.npy  
  inflating: full_grams_sg_300_twitter.mdl.wv.vectors.npy  


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/amrm3lm/cyborg-devs/main/combined_cleaned_data.csv")
df

,Unnamed: 0,raw_text,spam,clean_text,num_hashtags,num_mentions,num_words,num_emojis,num_links,num_unique_emojis,num_excessive_repeated_chars
0,0,الارتفاعات المستمرة في البيتكوين تظهر قوة وثبا...,1,"['الارتفاعات', 'المستمره', 'البيتكوين', 'تظهر'...",0,0,14,0,1,0,0
1,1,مالذي دفعه لقتل شقيقه؟\r\r\nأهو المال أم الحب؟...,1,"['مالذي', 'دفعه', 'لقتل', 'شقيقه', 'اهو', 'الم...",0,0,10,0,2,0,0
2,2,صعقو أهل المنزل عندما علمو أن خادمتهم ستشتريه ...,1,"['صعقو', 'اهل', 'المنزل', 'علمو', 'خادمتهم', '...",0,0,17,0,1,0,0
3,3,اتجاه المستثمرين لشراء البيتكوين وباقي العملات...,1,"['اتجاه', 'المستثمرين', 'لشراء', 'البيتكوين', ...",0,0,16,0,1,0,0
4,4,ثابرت وعملت بذكاء حتى استطاعت هذه الخادمة الو...,1,"['ثابرت', 'وعملت', 'بذكاء', 'استطاعت', 'الخادم...",0,0,13,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...
90543,90543,السحب الليلة على الايفون .. رتويت للمرفقة وطبق...,0,"['السحب', 'اليله', 'الايفون', 'رتويت', 'لمرفقه...",0,0,10,1,0,1,0
90544,90544,😂 لابسة احمر ليه يا ست انتي ايه المناسبة 😂,0,"['لابسه', 'احمر', 'ليه', 'انتي', 'المناسبه']",0,0,7,2,0,2,0
90545,90545,كلاام جمييل تستاهل(من احبه الله جعل محبته ف قل...,0,"['كلام', 'جميل', 'تستاهلمن', 'احبه', 'محبته', ...",0,0,8,1,0,1,0
90546,90546,- ألطف صورة ممكن تعبر عن رمضان 💙,0,"['الطف', 'صوره', 'مكن', 'تعبر', 'رمضان']",0,0,6,1,0,1,0


In [ ]:
y = df['spam']
X = df.drop(columns = ['spam', 'raw_text', 'Unnamed: 0'])

In [ ]:
# loading the AraVec word2vec model

model_w2v = gensim.models.Word2Vec.load('/content/full_grams_sg_300_twitter.mdl')

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(X['clean_text'])
vocab_size = len(t.word_index) + 1

# encoding the tweets
encoded_tweets = t.texts_to_sequences(X['clean_text'])

# padding the tweets
padded_tweets = pad_sequences(encoded_tweets, padding='post')

In [ ]:
padded_tweets

array([[ 24724,  13327,    915, ...,      0,      0,      0],
       [  4764,   3341,  16277, ...,      0,      0,      0],
       [ 55504,    364,   2286, ...,      0,      0,      0],
       ...,
       [   191,     87, 112152, ...,      0,      0,      0],
       [  1468,    159,    105, ...,      0,      0,      0],
       [   366,    112,   1423, ...,      0,      0,      0]], dtype=int32)

In [ ]:
w2v_embeddings_index = {}
TOTAL_EMBEDDING_DIM = 300
embeddings_file = '/content/full_grams_sg_300_twitter.mdl'
w2v_model = KeyedVectors.load(embeddings_file)
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocab_size, TOTAL_EMBEDDING_DIM))

for word, i in t.word_index.items():
    embedding_vector = w2v_embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print("Embedding Matrix shape:", embedding_matrix.shape)

Embedding Matrix shape: (112153, 300)


In [ ]:
# making an embedding layer to be used in lstm
embedding_layer = tf.keras.layers.Embedding(vocab_size, TOTAL_EMBEDDING_DIM, weights=[embedding_matrix], trainable=False)

In [ ]:
# model initializing

# importing libraries for the model
from keras.layers import Dense
from keras.layers import TimeDistributed, Bidirectional
from keras.layers import LSTM
from keras import Model, Input


input_placeholder = tf.keras.Input(shape=(1023,), dtype='int32')
input_embedding = embedding_layer(input_placeholder)
lstm= tf.keras.layers.LSTM(units=10, activation='relu')(input_embedding)
preds = tf.keras.layers.Dense(1, activation='sigmoid', name = 'activation')(lstm)
model = tf.keras.models.Model(inputs=input_placeholder, outputs=preds)

# compile the model
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999), metrics=['accuracy'])
# summarize the model
print(model.summary())


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1023)]            0         
                                                                 
 embedding (Embedding)       multiple                  33645900  
                                                                 
 lstm_3 (LSTM)               (None, 10)                12440     
                                                                 
 activation (Dense)          (None, 1)                 11        
                                                                 
Total params: 33,658,351
Trainable params: 12,451
Non-trainable params: 33,645,900
_________________________________________________________________
None


In [ ]:
loss, accuracy = model.evaluate(padded_tweets, y, verbose=1)

2830/2830 [==============================] - 377s 133ms/step - loss: 0.6931 - accuracy: 0.5000


In [ ]:
model.save("/content/lstm.keras")

In [ ]:
from sklearn.naive_bayes import MultinomialNB
# using naive bayes model as a benchmark

nbayes_model = MultinomialNB(alpha=0.01)
nbayes_model.fit(X_train, y_train)
nbayes_model.score(X_test, y_test)

0.8736163249590914

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# random forest

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.score(X_test, y_test)


0.865819616902493

In [ ]:
! pip install xgboost


In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
model.score(X_test, y_test)

0.7987852015461071

In [ ]:
y_pred = model.predict(X_test)
metrics.accuracy_score(y_test,y_pred )

0.7987852015461071

In [ ]:
model.save_model('xgboost.json')